In [1]:
pip install transformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
%pip install accelerate peft bitsandbytes transformers trl

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset,Dataset
import transformers

In [6]:
from random import randrange
dataset = load_dataset("Yijia-Xiao/pii-wikidoc_patient_information",split="train")

print(f"dataset size: {len(dataset)}")

# 'randrange' is a function from the 'random' module that generates a random number within the specified range.
print(dataset[randrange(len(dataset))])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5942 [00:00<?, ? examples/s]

dataset size: 5942
{'output': 'Call for an appointment with the health care provider or ophthalmologist if vision problems worsen, or do not improve with glasses or contact lenses.', 'instruction': 'Answer this question truthfully', 'input': 'When to seek urgent medical care when I have Astigmatism (eye) ?', 'cleaned_output': 'Call for an appointment with the health care provider or ophthalmologist if vision problems worsen, or do not improve with glasses or contact lenses.'}


In [7]:
dataset = dataset.remove_columns(['cleaned_output'])
dataset

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 5942
})

In [8]:
df = pd.DataFrame(dataset)


df = df[['instruction', 'input', 'output']]


rearranged_dataset = Dataset.from_pandas(df)
rearranged_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5942
})

In [9]:
model_name="distilbert/distilgpt2"

In [10]:
tokenizerr = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
tokenizer_id = model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id,)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token


In [12]:
# 'create_message_column' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'messages' and the value is a list of dictionaries, each representing a message.

# 'row' is the input to the 'create_message_column' function. It is expected to be a dictionary with keys 'instruction', 'input', and 'output'.

# 'messages' is a list that will contain the messages.

# 'user' is a dictionary that represents a user message. The 'content' key contains the instruction and input from the row, and the 'role' key is set to 'user'.

# 'messages.append(user)' adds the user message to the 'messages' list.

# 'assistant' is a dictionary that represents an assistant message. The 'content' key contains the output from the row, and the 'role' key is set to 'assistant'.

# 'messages.append(assistant)' adds the assistant message to the 'messages' list.

# 'return {"messages": messages}' returns a dictionary with a single key-value pair. The key is 'messages' and the value is the 'messages' list.

# 'format_dataset_chatml' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row.

# 'row' is the input to the 'format_dataset_chatml' function. It is expected to be a dictionary with a key 'messages'.

# 'return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}' returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row. The 'add_generation_prompt' parameter is set to False, which means that no generation prompt will be added to the end of the text. The 'tokenize' parameter is set to False, which means that the text will not be tokenized.
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruction']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

# def format_dataset_chatml(row):
#     return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


def format_dataset_chatml(row):
    text = tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)
    input_ids = tokenizer(text, padding='max_length', truncation=True, max_length=1024, return_tensors='pt').input_ids  # Adjust max_length
    return {"input_ids": input_ids}

In [13]:
dataset_chatml = rearranged_dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/5942 [00:00<?, ? examples/s]

Map:   0%|          | 0/5942 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


In [14]:
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'messages', 'input_ids'],
        num_rows: 5644
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'messages', 'input_ids'],
        num_rows: 298
    })
})

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

tokenizer

GPT2TokenizerFast(name_or_path='distilbert/distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [18]:
from peft import LoraConfig
from trl import SFTTrainer
import os
import torch
from transformers import (
    BitsAndBytesConfig,
    logging,
)

In [19]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,

)

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [21]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_aff

In [22]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"],
)

In [23]:
training_params = transformers.TrainingArguments(
    output_dir="/content/drive/MyDrive/finetuning/distilbert distilgpt2",
    num_train_epochs=3,
    do_eval=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=25,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    peft_config=peft_params,
    dataset_text_field="input_ids",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn

In [25]:
trainer.train()

Step,Training Loss,Validation Loss
25,4.087800,3.722773
50,3.892000,3.558116
75,3.769800,3.443388
100,3.634100,3.330776
125,3.549800,3.213056
150,3.347800,3.117676
175,3.361900,3.075593
200,3.257400,3.045956
225,3.274200,3.010100
250,3.225200,2.981415


TrainOutput(global_step=4233, training_loss=2.9047403310788824, metrics={'train_runtime': 5793.2746, 'train_samples_per_second': 2.923, 'train_steps_per_second': 0.731, 'total_flos': 4761753904742400.0, 'train_loss': 2.9047403310788824, 'epoch': 3.0})

In [26]:
trainer.save_model()

In [27]:
tokenizer.save_pretrained("/content/drive/MyDrive/finetuning/distilbert distilgpt2")

('/content/drive/MyDrive/finetuning/distilbert distilgpt2/tokenizer_config.json',
 '/content/drive/MyDrive/finetuning/distilbert distilgpt2/special_tokens_map.json',
 '/content/drive/MyDrive/finetuning/distilbert distilgpt2/vocab.json',
 '/content/drive/MyDrive/finetuning/distilbert distilgpt2/merges.txt',
 '/content/drive/MyDrive/finetuning/distilbert distilgpt2/added_tokens.json',
 '/content/drive/MyDrive/finetuning/distilbert distilgpt2/tokenizer.json')

In [33]:
hf_adapter_repo="ajsal27/dgpt2-medicalchatbot"
trainer.model.push_to_hub(hf_adapter_repo)

adapter_model.safetensors:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/dgpt2-medicalchatbot/commit/14a9aa8fe2d3d4d9358eff02bf5533401e9d334f', commit_message='Upload model', commit_description='', oid='14a9aa8fe2d3d4d9358eff02bf5533401e9d334f', pr_url=None, pr_revision=None, pr_num=None)

In [35]:
trainer.push_to_hub('ajsal27/chatbot')

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'distilbert distilgpt2'.

In [36]:
notebook_login()

In [37]:
trainer.model.push_to_hub("ajsal27/chatbot")

adapter_model.safetensors:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/chatbot/commit/b96a3059a90a53ca34c9ee119e4688a44d7edc87', commit_message='Upload model', commit_description='', oid='b96a3059a90a53ca34c9ee119e4688a44d7edc87', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
trainer.push_to_hub("ajsal27/dgpt2-medical-chatbot")

NameError: name 'trainer' is not defined

In [39]:
from transformers import pipeline

# Load the model from Hugging Face Hub
model_pipeline = pipeline("text-generation", model="ajsal27/chatbot")


adapter_config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [40]:
output = model_pipeline("symptoms of allergy")
print(output)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'symptoms of allergy include extreme and severe asthma, respiratory failure, dysentery, allergic reactions, and dermatitis, allergy to food.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]


In [41]:
from huggingface_hub import HfApi, Repository

# Create an API object
api = HfApi()

# Create a repository object (you need to clone it first if it does not exist)
repo_id = "ajsal27/chatbot"
repo = Repository(local_dir="/content/drive/MyDrive/finetuning/distilbert distilgpt2", repo_id=repo_id, repo_type="model")

# Push the tokenizer directory to the Hub
repo.push_to_hub(commit_message="Add tokenizer")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)


TypeError: Repository.__init__() got an unexpected keyword argument 'repo_id'

In [42]:
tokenizer.push_to_hub("ajsal27/chatbot", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ajsal27/chatbot/commit/6553e1ad3c2f9897d06f14e946afe48f719f25e5', commit_message='Upload tokenizer', commit_description='', oid='6553e1ad3c2f9897d06f14e946afe48f719f25e5', pr_url=None, pr_revision=None, pr_num=None)

In [43]:
output = model_pipeline("symptoms of allergy")
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'symptoms of allergy are usually mild and often moderate, but they include a slight increase in the number of symptoms with higher exposure.1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]


In [44]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer from the Hugging Face Hub
model = AutoModelForCausalLM.from_pretrained("ajsal27/chatbot")
tokenizer = AutoTokenizer.from_pretrained("ajsal27/chatbot")

# Define the input query
query = "What are the symptoms of allergy?"

# Tokenize the input query
inputs = tokenizer(query, return_tensors="pt")

# Generate text
with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,  # Adjust based on expected response length
        num_return_sequences=1,  # Number of responses to generate
        pad_token_id=tokenizer.eos_token_id,  # Handle padding tokens
        temperature=0.7,  # Adjust for creativity vs. coherence
        top_p=0.9,  # Use nucleus sampling for diverse outputs
        top_k=50  # Limits the sampling to the top-k tokens
    )

# Decode the generated response
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the response
print(generated_text)


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What are the symptoms of allergy?

The symptoms may include:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic reactions to food, such as:
Chronic allergic


In [45]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to the directory where your model and tokenizer are saved
model_path = "/content/drive/MyDrive/finetuning/distilbert-distilgpt2"
tokenizer_path = model_path  # If the tokenizer is in the same directory

# Load the model and tokenizer from the local directory
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Define the input query
query = "What are the symptoms of allergy?"

# Tokenize the input query
inputs = tokenizer(query, return_tensors="pt")

# Generate text
with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,  # Adjust based on expected response length
        num_return_sequences=1,  # Number of responses to generate
        pad_token_id=tokenizer.eos_token_id,  # Handle padding tokens
        temperature=0.7,  # Adjust for creativity vs. coherence
        top_p=0.9,  # Use nucleus sampling for diverse outputs
        top_k=50  # Limits the sampling to the top-k tokens
    )

# Decode the generated response
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the response
print(generated_text)


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/finetuning/distilbert-distilgpt2'. Please provide either the path to a local folder or the repo_id of a model on the Hub.